In [ ]:
import string

# Specify the path to the text file
file_path = 'data\smiles_ids.txt'
# Initialize an empty list to store the SMILES strings
smiles_dict = {}
# Open the file and read each line
with open(file_path, 'r') as file:
    for line in file:
        smiles, id = line.split(" ")
        smiles = smiles.strip()
        id = id.strip()
        smiles_dict[id] = smiles

print(len(smiles_dict))

# Define what counts as an atom (letters only)
def count_atoms(smiles):
    return sum(1 for c in smiles if c.isalpha())

# Filter the dictionary
smiles_dict = {id: smi for id, smi in smiles_dict.items() if count_atoms(smi) >= 5}
smiles_dict = {key: value.replace('.', '') for key, value in smiles_dict.items()}

print(len(smiles_dict))


246
236


In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoAlertPresentException
from selenium.common.exceptions import WebDriverException
import pathlib as p
import csv
import time
import shutil
import os
import random

#chrome_path = p.Path(r"C:\Users\usuario\Documents\vitoria\usp-masters\data\stp\chromedriver-win64\\chromedriver.exe").absolute()
#user_data = p.Path(r"C:\\Users\\usuario\\Documents\\vitoria\\usp-masters\\data\\stp\\server").absolute()
#download_path = p.Path(r"C:\\Users\\usuario\\Documents\\vitoria\\usp-masters\\data\\stp").absolute()

#chrome_path = p.Path(r"C:\\Users\\jvsam\\OneDrive\\Documentos\\masters\\usp-masters\\data\\stp\\chromedriver-win64\\chromedriver.exe").absolute()
#user_data = p.Path(r"C:\\Users\\jvsam\\OneDrive\\Documentos\\masters\\usp-masters\\data\\stp\\server").absolute()
#download_path = p.Path(r"C:\\Users\\jvsam\\OneDrive\\Documentos\\masters\\usp-masters\\data\\stp").absolute()

chrome_path = p.Path(r"C:\\Users\\vitor\\Documents\\usp\\usp-masters\\data\\stp\\chromedriver-win64\\chromedriver.exe").absolute()
user_data = p.Path(r"C:\\Users\\vitor\\Documents\\usp\\usp-masters\\data\\stp\\server").absolute()
download_path = p.Path(r"C:\\Users\\vitor\\Documents\\usp\\usp-masters\\data\\stp").absolute()

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=' + str(user_data))

service = Service(chrome_path)
prefs = {
    "download.default_directory": str(download_path),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("prefs", prefs)

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15"
]
options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")

driver = webdriver.Chrome(service=service, options=options)  

def is_browser_open(driver):
    try:
        # A simple command to test if browser is still reachable
        driver.title
        return True
    except WebDriverException:
        return False
    
def human_delay(min_sec=10, max_sec=20):
    time.sleep(random.uniform(min_sec, max_sec))

def predict_targets_stf(smiles, output_file, retries=3):
    for attempt in range(retries):
        try:
            driver.get('http://www.swisstargetprediction.ch/')
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#smilesBox'))).click()
            driver.find_element(By.CSS_SELECTOR, '#smilesBox').send_keys(smiles)    
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#submitButton'))).click()
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "button.dt-button.buttons-csv"))
            ).click()

            timeout = 30
            downloaded_file = None
            for _ in range(timeout):
                for filename in os.listdir(download_path):
                    if filename.endswith("SwissTargetPrediction.csv"):
                        downloaded_file = os.path.join(download_path, filename)
                        break
                if downloaded_file:
                    break
                time.sleep(1)

            if downloaded_file:
                final_path = os.path.join(download_path, output_file)
                shutil.move(downloaded_file, final_path)
                return
            else:
                raise Exception("Download failed or timed out.")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            human_delay(10, 30)
    print(f"Failed to process SMILES: {smiles}")

for idx, (id, smiles) in enumerate(filtered_dict.items()):
    if not is_browser_open(driver):
        print("Browser was closed. Exiting script.")
        break
    
    print(f"Processing {id} ({idx + 1}/{len(smiles_dict)})...")
    output_file = f"targets_{id}.csv"
    print(smiles)
    predict_targets_stf(smiles, output_file)
    human_delay(15, 30)

    # Restart browser every N iterations
    if (idx + 1) % 10 == 0:
        driver.quit()
        time.sleep(60)  # Cooldown period
        driver = webdriver.Chrome(service=service, options=options)

driver.quit()

Processing cid442424 (1/236)...
COC(=O)C1=COC(C2C1CC=C2CO)O
Attempt 1 failed: Alert Text: No similar actives found. No target predicted.
Message: unexpected alert open: {Alert text : No similar actives found. No target predicted.}
  (Session info: chrome=135.0.7049.85)
Stacktrace:
	GetHandleVerifier [0x00007FF6577C1F55+78133]
	GetHandleVerifier [0x00007FF6577C1FB0+78224]
	(No symbol) [0x00007FF6575891BA]
	(No symbol) [0x00007FF657599C29]
	(No symbol) [0x00007FF6575A08BA]
	(No symbol) [0x00007FF65757509D]
	(No symbol) [0x00007FF657599101]
	(No symbol) [0x00007FF65762F0F9]
	(No symbol) [0x00007FF657606F33]
	(No symbol) [0x00007FF6575D0358]
	(No symbol) [0x00007FF6575D10C3]
	GetHandleVerifier [0x00007FF657A8BA8D+3001453]
	GetHandleVerifier [0x00007FF657A85E72+2977874]
	GetHandleVerifier [0x00007FF657AA497D+3103581]
	GetHandleVerifier [0x00007FF6577DC7EA+186826]
	GetHandleVerifier [0x00007FF6577E43FF+218591]
	GetHandleVerifier [0x00007FF6577C9D94+110452]
	GetHandleVerifier [0x00007FF6577C9

KeyboardInterrupt: 

In [ ]:
import os

# Caminhos dos arquivos e diretórios
smiles_ids_file = "data/smiles_ids.txt"
csv_dir = "data/target_prediction/stp/"

# Lendo os IDs do arquivo smiles_ids.txt
with open(smiles_ids_file, "r") as file:
    ids = [line.strip().split()[1] for line in file]

# Verificando quais IDs não têm arquivos CSV
missing_ids = [id_ for id_ in ids if not os.path.exists(os.path.join(csv_dir, f"{id_}.csv"))]

# Imprimindo os IDs ausentes
print("IDs sem arquivo CSV:")
print("\n".join(missing_ids))

# Criando os arquivos CSV para os IDs ausentes
for id_ in missing_ids:
    with open(os.path.join(csv_dir, f"{id_}.csv"), "w") as file:
        file.write("No targets predicted.\n")

print("Arquivos CSV criados para os IDs ausentes.")


IDs sem arquivo CSV:
cid1146
cid175
cid702
cid283
cid223
cid280
cid1031
cid783
nocid1
nocid2
nocid3
nocid4
nocid5
nocid12
nocid13
nocid14
nocid15
nocid32
nocid33
nocid34
Arquivos CSV criados para os IDs ausentes.
